# Creating an NSE table

We can tabulate the NSE state on a grid $(\rho, T, Y_e)$, giving:

* $\bar{A}$ : the mean molecular weight
* $\langle B/A \rangle$ : the average binding energy per nucleon of the NSE state [MeV]
* $dY_e/dt$ : the time-evolution of $Y_e$ due to weak reactions [1/s]
* $d\bar{A}/dt$ : the time-evolution of $\bar{A}$ due to weak reactions [1/s]
* $d\langle B/A\rangle/dt$ : the time-evolution of the binding energy per nucleon due to weak reactions [MeV/s]
* $\epsilon_\nu$ : the energy loss due to neutrinos [erg / g / s] 

In [1]:
import pynucastro as pyna
from pynucastro import Nucleus

First we'll create a simple NSE network with a few nuclei.  We'll include tabulated weak rates so we can compute the change in the state due to electron/positron captures and decays.

In [2]:
nucs = [Nucleus("p"), Nucleus("n"), Nucleus("he4"),
        Nucleus("fe52"), Nucleus("fe53"), Nucleus("fe54"),
        Nucleus("fe55"), Nucleus("fe56"),
        Nucleus("co54"), Nucleus("co55"), Nucleus("co56"),
        Nucleus("ni56"), Nucleus("ni57")]

In [3]:
tl = pyna.TabularLibrary().linking_nuclei(nucs)
rl = pyna.ReacLibLibrary().linking_nuclei(nucs)
all_lib = tl + rl

In [4]:
dupes = all_lib.find_duplicate_links()
rates_to_remove = []
for d in dupes:
    rates_to_remove += [r for r in d if isinstance(r, pyna.rates.ReacLibRate)]

for r in rates_to_remove:
    all_lib.remove_rate(r)

In [5]:
nse = pyna.NSENetwork(libraries=[all_lib])

Now we'll create a grid of temperature, density, and $Y_e$ where we want to compute the NSE state

In [6]:
import numpy as np

In [7]:
Ts = np.logspace(9.6, 10.4, 3)
rhos = np.logspace(7, 10, 4)
yes = np.linspace(0.43, 0.5, 3)

Finally we can generate the table.  This will compute the NSE state at each combination of $(\rho, T, Y_e)$.  To help accelerate the convergence,
it will start at the highest temperature and loop over $\rho$ and $Y_e$ and cache the values of the proton and neutron chemical potentials for the
next temperature.

In [8]:
nse.generate_table(rho_values=rhos,
                   T_values=Ts,
                   Ye_values=yes)

The table is stored as `nse.tbl`

In [9]:
%cat nse.tbl

# NSE table generated by pynucastro 2.3.0.post13+g3d9952d0
# original NSENetwork had 13 nuclei
#
#   log10(rho)       log10(T)           Ye             Abar            <B/A>          dYe/dt         dAbar/dt        d<B/A>/dt         e_nu       
   7.0000000000    9.6000000000    0.5000000000   50.2051965167    8.6280555771  -4.3798684e-05              -0   2.3285948e-06   7.4765875e+13 
   7.0000000000    9.6000000000    0.4650000000   55.7461181559    8.7870631885  -5.5253373e-09  -1.1145835e-21   3.9317237e-10   1.3980145e+10 
   7.0000000000    9.6000000000    0.4300000000   11.0637644863    8.1412181124   0.00017699678  -2.9553154e-26  -1.8453759e-12   2.7707299e+14 
   7.0000000000   10.0000000000    0.5000000000    1.2148721368    1.6682000987     0.084056593   -6.583548e-57   5.7350248e-41   7.8635828e+17 
   7.0000000000   10.0000000000    0.4650000000    1.2129154977    1.6556759493      0.10066493  -5.9529967e-56   3.5220152e-41   8.3544455e+17 
   7.0000000000   10.0000000000

There are other options to the NSE table writing, including storing the mass fractions or a reduced composition constructed by binning the nuclei
into the closest nuclei in a smaller set.